## AAPL StockPricing Prediction

### Import Libraries

In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import yfinance as yf

import pandas_datareader as pdr
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from xgboost import XGBRegressor

from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

c:\Users\simon.cerda\Anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### APPL Stock Data

In [2]:

######################################################################################
stock_data = yf.download('AAPL') # Download APPL stock data

num_days_pred=30 # Number of days you want to predict in the future the higher the less accuracy
######################################################################################

# Here I choose to only use the last 3 years of stock data 
slice = int(len(stock_data)- 356*3)
stock_data = stock_data.iloc[slice:]

# Here I Choose to continue with only Close value column since that is what we care about 
stock_data.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'],inplace=True)


# Function to calculate mean absolute error percentage
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def add_lags(df,num_days_pred=num_days_pred):
    target = 'Close'
    df['lag1'] = df[target].shift(num_days_pred)  
    df['lag2'] = df[target].shift(num_days_pred*2)    
    df['lag3'] = df[target].shift(num_days_pred*3)    
    df['lag4'] = df[target].shift(num_days_pred*4)    
    df['lag5'] = df[target].shift(num_days_pred*5)
    df['lag6'] = df[target].shift(num_days_pred*6)
    df['lag7'] = df[target].shift(num_days_pred*7)
    df['lag8'] = df[target].shift(num_days_pred*8)
    df['lag9'] = df[target].shift(num_days_pred*9)
    df['lag10'] = df[target].shift(num_days_pred*10)
    df['lag11'] = df[target].shift(num_days_pred*11)
    df['lag12'] = df[target].shift(num_days_pred*12)



    return df

def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

[*********************100%%**********************]  1 of 1 completed


# XGBOOST

In [3]:
df_xgb = stock_data.copy()

In [4]:
def xgboostmodel(df_xgb,add_lags,create_features,num_days_pred=num_days_pred):

    df_xgb = create_features(df_xgb)
    df_xgb = add_lags(df_xgb)
    
    X = df_xgb.drop(columns='Close')
    y = df_xgb['Close']
    return X,y
X,y = xgboostmodel(df_xgb ,add_lags,create_features,num_days_pred=30)

In [5]:
# Define objective function for Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
def objective(trial):
    # Define hyperparameters to search
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'verbosity': 0,
        #'tree_method': 'gpu_hist',
    }
    
    # Initialize XGBoost regressor with the suggested parameters
    xgb = XGBRegressor(**param)
    
    # Fit the model on training data
    xgb.fit(X_train, y_train)
    
    # Predict on the validation set
    y_pred = xgb.predict(X_test)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return rmse


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Perform hyperparameter optimization using Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best trial and parameters found
print("Best trial:")
best_trial = study.best_trial
print(f"  Value: {best_trial.value}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# Use the best parameters to train the final model
best_params = best_trial.params
xgb_best = XGBRegressor(**best_params)
xgb_best.fit(X_train, y_train,verbose=False)

# Make predictions on the test set
y_pred_test = xgb_best.predict(X_test)

# Calculate RMSE on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print("Test RMSE:", rmse_test)

Best trial:
  Value: 3.258836452973074
  Params: 
    n_estimators: 963
    max_depth: 10
    learning_rate: 0.07975803332551062
    subsample: 0.7329416495919234
    colsample_bytree: 0.6917664437681972
    reg_alpha: 7.625232272087433
    reg_lambda: 4.322000961443669
Test RMSE: 3.258836452973074


## Evaluate

In [6]:
y_pred_test_xgb = xgb_best.predict(X_test)
xgb_loss = mean_absolute_percentage_error(y_test, y_pred_test_xgb) 
print(f"ERROR PERCENT = { mean_absolute_percentage_error(y_test, y_pred_test_xgb) }% ")

ERROR PERCENT = 1.7773427235475343% 


In [7]:
feature_importances = xgb_best.feature_importances_

# To get the feature names
feature_names = X_train.columns

# Create a pandas series to visualize the feature importances
importance_series = pd.Series(data=feature_importances, index=feature_names)

# Sort the feature importances
print(importance_series.sort_values(ascending=False))

year          0.508546
lag9          0.247954
lag12         0.079842
lag8          0.046287
lag1          0.033997
lag2          0.016013
lag4          0.013664
lag3          0.012468
lag11         0.007359
weekofyear    0.006593
lag7          0.005859
month         0.005657
dayofyear     0.005425
quarter       0.004387
lag5          0.002257
lag10         0.002153
lag6          0.000946
dayofmonth    0.000393
dayofweek     0.000201
hour          0.000000
dtype: float32


# Prophet

In [8]:
df_prophet = stock_data.copy()

Split

In [9]:
split_date = df_prophet.index[int(len(df_prophet) * 0.8)]
train = df_prophet.loc[df_prophet.index <= split_date].copy()
test = df_prophet.loc[df_prophet.index > split_date].copy()

Preprocess

In [10]:
# Format data for prophet model using ds and y
train_prophet = train.reset_index() \
    .rename(columns={'Date':'ds',
                     'Close':'y'})

In [11]:
train_prophet 

,ds,y
0,2020-02-05,80.362503
1,2020-02-06,81.302498
2,2020-02-07,80.007500
3,2020-02-10,80.387497
4,2020-02-11,79.902496
...,...,...
850,2023-06-22,187.000000
851,2023-06-23,186.679993
852,2023-06-26,185.270004
853,2023-06-27,188.059998


Training

In [12]:
prophet = Prophet()
prophet.fit(train_prophet)

18:56:00 - cmdstanpy - INFO - Chain [1] start processing
18:56:01 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
# Format data for prophet model using ds and y
test_prophet = test.reset_index() \
    .rename(columns={'Date':'ds',
                     'Close':'y'})
test_predict = prophet.predict(test_prophet)

Evaluating

In [14]:
porphet_loss = mean_absolute_percentage_error(test['Close'],test_predict['yhat'] )
print(f"ERROR PERCENT = { mean_absolute_percentage_error(test['Close'],test_predict['yhat'] ) }% ")

ERROR PERCENT = 19.93958908909836% 


# ARIMA

In [15]:
df_arima = stock_data.copy()

Split

In [16]:
split_date = df_prophet.index[int(len(df_arima) * 0.8)]
train_arima = df_arima.loc[df_arima.index <= split_date].copy()
test_arima = df_arima.loc[df_arima.index > split_date].copy()

hyperparamter tuning 

In [17]:
# Try to find the best parameters for arima model 
stepwise_fit = auto_arima(train_arima['Close'],trace=True,suppress_warnings=True)
# assign the parameter to "best_order" variable
best_order = stepwise_fit.get_params()['order']
print(best_order)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=4148.921, Time=0.55 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4145.049, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=4144.158, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4143.969, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4144.905, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4144.935, Time=0.16 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=4145.170, Time=0.12 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=4146.899, Time=0.42 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=4144.075, Time=0.03 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0] intercept
Total fit time: 1.459 seconds
(0, 1, 1)


train

In [18]:
arima = ARIMA(train_arima['Close'], order=best_order)
arima = arima.fit()

evaluate

In [19]:
start = len(train_arima)
end = len(test_arima) + len(train_arima)

In [20]:
pred_arima = arima.predict(start=start,end=end-1)

In [21]:
pred_arima.index = test_arima.index

In [22]:
arima_loss = mean_absolute_percentage_error(test_arima['Close'],pred_arima )
print(f"ERROR PERCENT = { mean_absolute_percentage_error(test_arima['Close'],pred_arima ) }% ")

ERROR PERCENT = 5.373443181845581% 


In [23]:
print(f"XGB Acc : {100-xgb_loss} \nArima Acc : {100-arima_loss}\nProphet Acc : {100- porphet_loss}")

XGB Acc : 98.22265727645247 
Arima Acc : 94.62655681815441
Prophet Acc : 80.06041091090164
